In [1]:
#Install elastic search
!pip install elasticsearch


Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install -q pandas openai

In [3]:
!docker-compose up -d

 elasticsearch Pulling 
 527f5363b98e Already exists 
 a88b906db9c1 Already exists 
 02840f0dd2d9 Already exists 
 4f4fb700ef54 Already exists 
 d36bf0362a7b Already exists 
 7cfad007f0ac Already exists 
 4bcb210dd3ca Already exists 
 a1bae0d7353f Already exists 
 2aa937b89ef3 Already exists 
 7132da5d7ea5 Already exists 
 90ae6586c1d5 Already exists 
 elasticsearch Pulled 
 Network pull_github_issues_default  Creating
 Network pull_github_issues_default  Created
 Container pull_github_issues-elasticsearch-1  Creating
 Container pull_github_issues-elasticsearch-1  Created
 Container pull_github_issues-elasticsearch-1  Starting
 Container pull_github_issues-elasticsearch-1  Started


In [14]:
# To search documents 

#Import Elascticsearch and helpers from  elasticsearch

from elasticsearch import Elasticsearch,helpers
import json

es = Elasticsearch(['http://localhost:9200'])
es.ping()   #connection testing
res = es.search(index="github_issues", body={"query": {"match_all": {}}})

# Extract the total hit count
print("Got %d Hits:" % res['hits']['total']['value'])

# Extract hits from the response
hits = res.get('hits', {})

# Pretty print the hits
print(json.dumps(hits, indent=2))


Got 751 Hits:
{
  "total": {
    "value": 751,
    "relation": "eq"
  },
  "max_score": 1.0,
  "hits": [
    {
      "_index": "github_issues",
      "_id": "IbObu44BpIZtBu2_LsdO",
      "_score": 1.0,
      "_source": {
        "_type": "issue",
        "_repo": "SCM587SP24",
        "_issueNumber": "832",
        "_title": "Fixes #830 ASP24SCM96P",
        "_createdAt": "2024-04-07T05:09:49Z",
        "_closedAt": "2024-04-07T05:11:32Z",
        "_state": "closed",
        "_body": "None",
        "GitHub_Issue_vector": [
          -0.010275030508637428,
          0.009742610156536102,
          0.03117082081735134,
          -0.0008548120385967195,
          -0.030811263248324394,
          0.012432372197508812,
          -0.02377224527299404,
          -0.023869048804044724,
          -0.029898541048169136,
          -0.02761673741042614,
          0.024864744395017624,
          -0.012370141223073006,
          -0.0034659213852137327,
          0.019512878730893135,
          -0.0

In [15]:
# Define model and embedding function
from openai import OpenAI
EMBEDDING_MODEL = "text-embedding-ada-002"
client = OpenAI(api_key="")
def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding


In [16]:
# Pretty print function for GitHub Issues

def pretty_response_Issue(response):
    for hit in response['hits']['hits']:
        type = hit['_source']['_type']
        repo =hit['_source']['_repo']
        issueNumber = hit['_source']['_issueNumber']
        title = hit['_source']['_title']
        createdAt = hit['_source']['_createdAt']
        closedAt = hit['_source']['_closedAt']
        state = hit['_source']['_state']
        body = hit['_source']['_body']
        
        pretty_output = (
            f"\nType: {type}\nRepo:{repo}\nIssue Number: {issueNumber}\nTitle: {title}\nCreated At: {createdAt}\nClosed At: {closedAt}\n"
            f"State: {state}\nBody: {body}\n"
        )
        print(pretty_output)

## Lets retrieve React Issues
- Notice React issues are listed first
- After that other Rendering issues are listed

In [17]:
#Searching based on githubissues index 
#This is an example for the semantic search.
# Define question
question = 'Give me the Issues that are from the react repository'

question_embedding = get_embedding(question, model=EMBEDDING_MODEL)

response = es.search(
  index="github_issues",
  knn={
      "field": "GitHub_Issue_vector",  
      "query_vector": question_embedding,
      "k": 5,
      "num_candidates": 100
    }
)

pretty_response_Issue(response)



Type: issue
Repo:SCM587SP24
Issue Number: 108
Title: React Repository Issue: Unable to Fetch API Data
Created At: 2024-03-19T17:01:27Z
Closed At: 2024-03-19T17:38:28Z
State: closed
Body: I am encountering an issue in my React repository where I am unable to fetch data from an API. I have checked the API endpoint and my fetch method, but I am still not receiving any data. This issue is preventing me from completing a crucial feature in my application. Any help in resolving this would be highly valued. Thank you.


Type: issue
Repo:SCM587SP24
Issue Number: 113
Title: React Repository Issue: Performance Degradation
Created At: 2024-03-19T17:42:32Z
Closed At: 2024-03-19T17:56:09Z
State: closed
Body: I am noticing a performance degradation in my React repository, especially when rendering large lists or components. I have tried optimizing my code and using techniques like memoization, but the performance issues persist. This issue is affecting the user experience of my application. Any sug

## Lets retrieve Rendering Issues
- Notice Rendering Issues are listed first
- After that other React issues are listed

In [18]:
#Searching based on githubissues index 
#This is an example for the semantic search.
# Define question
question = 'Give me a list of Rendering Issues created for React repository'

question_embedding = get_embedding(question, model=EMBEDDING_MODEL)

response = es.search(
  index="github_issues",
  knn={
      "field": "GitHub_Issue_vector",  
      "query_vector": question_embedding,
      "k": 5,
      "num_candidates": 100
    }
)

pretty_response_Issue(response)



Type: issue
Repo:SCM587SP24
Issue Number: 113
Title: React Repository Issue: Performance Degradation
Created At: 2024-03-19T17:42:32Z
Closed At: 2024-03-19T17:56:09Z
State: closed
Body: I am noticing a performance degradation in my React repository, especially when rendering large lists or components. I have tried optimizing my code and using techniques like memoization, but the performance issues persist. This issue is affecting the user experience of my application. Any suggestions on how to improve performance would be highly valued. Thank you.


Type: issue
Repo:SCM587SP24
Issue Number: 572
Title: Improve Performance of Component Rendering
Created At: 2024-03-29T00:46:32Z
Closed At: 2024-03-29T04:50:25Z
State: closed
Body: Currently, there are performance bottlenecks in rendering large components within our React application. This issue aims to address these inefficiencies and optimize the rendering process for better performance. We need to analyze the component structure, identi

## Lets retrieve OpenAI-cookbook Issues
- Notice OpenAI-cookbook issues are listed first
- After that other Rate-limit issues are listed

In [19]:
#Searching based on githubissues index 
#This is an example for the semantic search.
# Define question
question = 'Give me the Issues that are from the OpenAI-cookbook repository'

question_embedding = get_embedding(question, model=EMBEDDING_MODEL)

response = es.search(
  index="github_issues",
  knn={
      "field": "GitHub_Issue_vector",  
      "query_vector": question_embedding,
      "k": 5,
      "num_candidates": 100
    }
)

pretty_response_Issue(response)


Type: issue
Repo:SCM587SP24
Issue Number: 347
Title: Improving Recipe Organization
Created At: 2024-03-26T21:38:17Z
Closed At: 2024-03-28T19:59:07Z
State: closed
Body: Currently, the organization of recipes in the OpenAI-cookbook repo could be enhanced for better usability and navigation. This issue aims to discuss and implement improvements to the structure and categorization of recipes within the repository. Some potential areas for improvement include creating subfolders for different cuisine types, adding tags to recipes for easier searching, and ensuring consistent formatting across all recipes. Community input and collaboration are welcome to make the OpenAI-cookbook a more user-friendly and organized resource for all users.


Type: issue
Repo:SCM587SP24
Issue Number: 677
Title: Request for Vegan/Vegetarian Recipes
Created At: 2024-03-29T19:16:43Z
Closed At: 2024-03-31T18:04:13Z
State: closed
Body: This issue is a request to add more vegan and vegetarian recipes to the OpenAI-co

## Lets retrieve Rate-limit Issues created for OpenAI-cookbook repository
- Notice Rate-limit Issues are listed first
- After that other OpenAI-cookbook issues are listed

In [20]:
#Searching based on githubissues index 
#This is an example for the semantic search.
# Define question
question = 'Give me a list of Rate-limit Issues created for OpenAI-cookbook repository'

question_embedding = get_embedding(question, model=EMBEDDING_MODEL)

response = es.search(
  index="github_issues",
  knn={
      "field": "GitHub_Issue_vector",  
      "query_vector": question_embedding,
      "k": 5,
      "num_candidates": 100
    }
)

pretty_response_Issue(response)


Type: issue
Repo:SCM587SP24
Issue Number: 136
Title: GitHub API Rate Limit Issues
Created At: 2024-03-23T21:35:22Z
Closed At: 2024-12-31T00:36:30Z
State: open
Body: Experiencing rate limit constraints while accessing GitHub API endpoints in the OpenAI-cookbook repository. Seeking solutions to mitigate rate limiting challenges.


Type: issue
Repo:SCM587SP24
Issue Number: 137
Title: Addressing Rate Limit Errors on GitHub API
Created At: 2024-03-23T21:35:53Z
Closed At: 2024-12-31T00:36:30Z
State: open
Body: Facing rate limit exceeded errors when making frequent requests to GitHub API within the OpenAI-cookbook repository. Looking for ways to optimize API usage and prevent throttling


Type: issue
Repo:SCM587SP24
Issue Number: 138
Title: Rate Limit Exceeded Error on Embeddings Generation
Created At: 2024-03-23T21:36:36Z
Closed At: 2024-12-31T00:36:30Z
State: open
Body: Encountering rate limit exceeded errors when generating embeddings in the OpenAI-cookbook repository. Need to address lim

In [ ]:
#Requirement 3

In [21]:
#Searching based on githubissues index 
#This is an example for the semantic search.
# Define question
question = 'Give me top 5 similar issues that are in the index of our repository.'

question_embedding = get_embedding(question, model=EMBEDDING_MODEL)

response = es.search(
  index="github_issues",
  knn={
      "field": "GitHub_Issue_vector",  
      "query_vector": question_embedding,
      "k": 5,
      "num_candidates": 100
    }
)

pretty_response_Issue(response)


Type: issue
Repo:SCM587SP24
Issue Number: 136
Title: GitHub API Rate Limit Issues
Created At: 2024-03-23T21:35:22Z
Closed At: 2024-12-31T00:36:30Z
State: open
Body: Experiencing rate limit constraints while accessing GitHub API endpoints in the OpenAI-cookbook repository. Seeking solutions to mitigate rate limiting challenges.


Type: issue
Repo:SCM587SP24
Issue Number: 133
Title: Improve integration with modern web frameworks
Created At: 2024-03-22T22:25:42Z
Closed At: 2024-12-31T00:36:30Z
State: open
Body: d3.js integration with modern web frameworks like React, Vue.js, and Angular could be improved. Providing official guides or libraries for integrating d3.js with these frameworks would make it easier for developers to use d3.js in their projects.


Type: issue
Repo:SCM587SP24
Issue Number: 668
Title: Optimize Codebase for Improved Performance
Created At: 2024-03-29T18:42:40Z
Closed At: 2024-03-31T18:01:53Z
State: closed
Body: The current codebase of the OpenAI-cookbook repository 

In [ ]:
#Requirement 4

In [22]:
#Searching based on githubissues index 
#This is an example for the semantic search.
# Define question


question = 'Give me top 5 similar Rendering React issues that are created in our repo.'

question_embedding = get_embedding(question, model=EMBEDDING_MODEL)

response = es.search(
  index="github_issues",
  knn={
      "field": "GitHub_Issue_vector",  
      "query_vector": question_embedding,
      "k": 5,
      "num_candidates": 100
    }
)

pretty_response_Issue(response)


Type: issue
Repo:SCM587SP24
Issue Number: 572
Title: Improve Performance of Component Rendering
Created At: 2024-03-29T00:46:32Z
Closed At: 2024-03-29T04:50:25Z
State: closed
Body: Currently, there are performance bottlenecks in rendering large components within our React application. This issue aims to address these inefficiencies and optimize the rendering process for better performance. We need to analyze the component structure, identify potential areas of improvement such as unnecessary re-renders, and implement strategies like memoization or shouldComponentUpdate to optimize rendering. Additionally, we should consider implementing virtualization techniques for large lists or grids to improve overall performance. Let's collaborate on finding and implementing solutions to enhance the performance of our React application.


Type: issue
Repo:SCM587SP24
Issue Number: 634
Title: Improve Performance of React Component Rendering
Created At: 2024-03-29T14:40:10Z
Closed At: 2024-03-31T17:

In [ ]:
#Requiremnt 5

In [23]:
#Searching based on githubissues index 
#This is an example for the semantic search.
# Define question


question = "Give me top 5 similar Rendering React issues that are created in our repo between 3/27/2024 and 3/29/2024."

question_embedding = get_embedding(question, model=EMBEDDING_MODEL)

response = es.search(
  index="github_issues",
  knn={
      "field": "GitHub_Issue_vector",  
      "query_vector": question_embedding,
      "k": 5,
      "num_candidates": 100,
      "filter": {
                "range": {
                "_createdAt": {
                "gte": "2024-03-27T00:00:00Z",
                "lte": "2024-03-29T23:59:59Z"
                        }
                    }
     }
    }
)

pretty_response_Issue(response)





Type: issue
Repo:SCM587SP24
Issue Number: 708
Title: Bug: Component Rendering Issue in React 18
Created At: 2024-03-29T22:35:40Z
Closed At: 2024-03-31T17:32:30Z
State: closed
Body: After upgrading to React 18, we've encountered a critical bug where certain components fail to render properly in our application. Despite no changes in the component code, some components intermittently fail to display or render incorrectly. This issue is impacting the user experience and causing frustration among our users. We've thoroughly reviewed our codebase and haven't identified any obvious causes within our application. Given the severity of the issue and its impact on our users, we request urgent attention from the React development team to investigate and provide a resolution. Any guidance or assistance in resolving this issue would be greatly appreciated.


Type: issue
Repo:SCM587SP24
Issue Number: 572
Title: Improve Performance of Component Rendering
Created At: 2024-03-29T00:46:32Z
Closed At: 

In [ ]:
#Requirement 5.1 (own query)

In [24]:
#Searching based on githubissues index 
#This is an example for the semantic search.
# Define question


question = "Give me top 5 similar Rendering React issues that are created in our repo between (3/27/2024 and 3/29/2024) and closed between(3/30/2024 and 4/02/2024)."

question_embedding = get_embedding(question, model=EMBEDDING_MODEL)

response = es.search(
  index="github_issues",
  knn={
      "field": "GitHub_Issue_vector",  
      "query_vector": question_embedding,
      "k": 5,
      "num_candidates": 100,
      "filter": {
                "range": {
                "_createdAt": {
                "gte": "2024-02-27T00:00:00Z",
                "lte": "2024-02-29T23:59:59Z"
                        }
                    },
                "range":{
                    "_closedAt":{
                    "gte": "2024-03-30T00:00:00Z",
                    "lte": "2024-04-02T23:59:59Z"
                }
                }
     }
    }
)

pretty_response_Issue(response)





Type: issue
Repo:SCM587SP24
Issue Number: 708
Title: Bug: Component Rendering Issue in React 18
Created At: 2024-03-29T22:35:40Z
Closed At: 2024-03-31T17:32:30Z
State: closed
Body: After upgrading to React 18, we've encountered a critical bug where certain components fail to render properly in our application. Despite no changes in the component code, some components intermittently fail to display or render incorrectly. This issue is impacting the user experience and causing frustration among our users. We've thoroughly reviewed our codebase and haven't identified any obvious causes within our application. Given the severity of the issue and its impact on our users, we request urgent attention from the React development team to investigate and provide a resolution. Any guidance or assistance in resolving this issue would be greatly appreciated.


Type: issue
Repo:SCM587SP24
Issue Number: 634
Title: Improve Performance of React Component Rendering
Created At: 2024-03-29T14:40:10Z
Close

## Sanity Test:

Execute following commands from the command window/terminal:

**To verify the index got created:**
- curl -XGET "http://localhost:9200/_cat/indices?v"

**To check the structure of index:**
- curl -X GET "localhost:9200/github_issues?pretty"

**To check the stats of the index:**

- curl -X GET "localhost:9200/github_issues/_stats?pretty"

**To dump the documents in the index to stdout:**
- curl -XGET "http://localhost:9200/github_issues/_search?pretty=true"

**To dump the mapping of  the index to stdout:**
- curl -XGET "localhost:9200/github_issues/_mapping?pretty"


## How to delete  the index?
**To delete the index you created:**
- curl -XDELETE "http://localhost:9200/github_issues?pretty"

